#property indicator_chart_window
#property indicator_buffers 1
#property indicator_plots   1

#property indicator_label1  "Tick-Volume Developing POC"
#property indicator_type1   DRAW_LINE
#property indicator_color1  clrOrange
#property indicator_width1  2

// ─────────────────────────────────────
input double PriceStepMult = 1.0;

double PocLine[];

// --- internal state
double step;
double pLow, pHigh;
double volMap[];
int    mapSize = 0;

long   lastTickVolume = 0;
bool   initialized = false;

// ─────────────────────────────────────
int OnInit()
{
   SetIndexBuffer(0, PocLine, INDICATOR_DATA);
   ArraySetAsSeries(PocLine, true);

   step = _Point * PriceStepMult;
   return INIT_SUCCEEDED;
}

// ─────────────────────────────────────
int OnCalculate(
   const int rates_total,
   const int prev_calculated,
   const datetime& time[],
   const double& open[],
   const double& high[],
   const double& low[],
   const double& close[],
   const long& tick_volume[],
   const long& volume[],
   const int& spread[]
)
{
   if(rates_total < 2)
      return prev_calculated;

   // --- initialize on first call
   if(!initialized)
   {
      double price = close[0];
      pLow = pHigh = price;

      mapSize = 1;
      ArrayResize(volMap, 1);
      volMap[0] = 0.0;

      lastTickVolume = tick_volume[0];
      PocLine[0] = price;

      initialized = true;
      return rates_total;
   }

   // --- detect new ticks on current bar
   long delta = tick_volume[0] - lastTickVolume;
   lastTickVolume = tick_volume[0];

   if(delta <= 0)
      return rates_total;

   double price = close[0];

   // --- expand profile range if needed
   if(price < pLow)
   {
      int add = int((pLow - price) / step) + 1;
      int old = mapSize;
      mapSize += add;

      ArrayResize(volMap, mapSize);
      for(int i = old; i < mapSize; i++)
         volMap[i] = 0.0;

      pLow -= add * step;
   }
   else if(price > pHigh)
   {
      int add = int((price - pHigh) / step) + 1;
      int old = mapSize;
      mapSize += add;

      ArrayResize(volMap, mapSize);
      for(int i = old; i < mapSize; i++)
         volMap[i] = 0.0;

      pHigh += add * step;
   }

   int idx = int((price - pLow) / step);
   if(idx >= 0 && idx < mapSize)
      volMap[idx] += (double)delta;

   // --- compute POC
   double maxVol = 0.0;
   int pocIdx = 0;

   for(int i = 0; i < mapSize; i++)
   {
      if(volMap[i] > maxVol)
      {
         maxVol = volMap[i];
         pocIdx = i;
      }
   }

   double pocPrice = pLow + pocIdx * step;

   // --- develop line
   PocLine[0] = pocPrice;
   if(prev_calculated > 0)
      PocLine[1] = PocLine[0];

   return rates_total;
}
